# Perspective Transform
  It is any combination of rotation, scaling, reflection or shearing on an image
 ![perspective](./Assets/getperspective_transform_01.jpg)
 
 It can be done by just a matrix multiplication of the coordinates with a transformation matrix
 
 Example for Rotation
 \begin{equation}
\left[\begin{array}{l}
x^{\prime} \\
y^{\prime} \\
1
\end{array}\right]=\left[\begin{array}{lll}
1 & 0 & t_{x} \\
0 & 1 & t_{y} \\
0 & 0 & 1
\end{array}\right]\left[\begin{array}{l}
x \\
y \\
1
\end{array}\right]
\end{equation}
 
![transform](./Assets/2D_affine_transformation_matrix.svg)
 

In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

billboard = cv2.imread('./Assets/billboard.png')
widthb, heightb = billboard.shape[1]//3, billboard.shape[0]//3
billboard = cv2.resize(billboard,(widthb, heightb))
billboard_copy = billboard.copy()

advert = cv2.imread('./Assets/advert.jpg')
widtha, heighta = advert.shape[1]//8, advert.shape[0]//8
advert = cv2.resize(advert, (widthb, heightb))

In [18]:
cv2.imshow('billboard',billboard)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
cv2.imshow('advt',advert)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
corner_points = []
def point_selection(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        corner_points.append([x,y])
        cv2.circle(billboard_copy, (x,y), 3, (0,255,0),-1)
        if len(corner_points)<=4:
            cv2.imshow('selection', billboard_copy)




In [21]:
cv2.imshow('selection', billboard_copy)
cv2.setMouseCallback('selection',point_selection)
cv2.waitKey(0)
cv2.destroyAllWindows()
corner_points = corner_points[:4]
corner_points = np.array(corner_points, dtype='float32')
print(corner_points)

[[ 62.  18.]
 [ 52. 208.]
 [375. 236.]
 [369.  88.]]


In [22]:
points_original = np.array([[0,0],[0,heightb], [widthb,heightb], [widthb,0]], dtype='float32')
M = cv2.getPerspectiveTransform(points_original, corner_points)
warped_advert = cv2.warpPerspective(advert, M, (widthb, heightb))
cv2.imshow('advert' , warped_advert)
cv2.waitKey(0)
cv2.destroyAllWindows()

Calculates coefficients of perspective transformation
which maps $(x_i,y_i)$ to $(u_i,v_i), (i=1,2,3,4)$:


$$
u_i = \frac{c_{00} \times x_i + c_{01} \times y_i + c_{02}}{c_{20} \times x_i + c_{21} \times y_i + c_{22}}
$$


$$
\mathrm{v}_i = \frac{c_{10} \times x_i + c_{11} \times y_i + c_{12}}{c_{20} \times x_i + c_{21} \times y_i + c_{22}}
$$


Coefficients are calculated using this linear system


$$
\left(
\begin{array}{rrrrrrrr}
x 0 & y 0 & 1 & 0 & 0 & 0 & -x 0 \times \mathrm{u} 0 & -y 0 \times \mathrm{u} 0 \\
x 1 & y 1 & 1 & 0 & 0 & 0 & -x 1 \times \mathrm{u} 1 & -y 1 \times \mathrm{u} 1 \\
x 2 & y 2 & 1 & 0 & 0 & 0 & -x 2 \times \mathrm{u} 2 & -y 2 \times \mathrm{u} 2 \\
x 3 & y 3 & 1 & 0 & 0 & 0 & -x 3 \times \mathrm{u} 3 & -y 3 \times \mathrm{u} 3 \\
0 & 0 & 0 & x 0 & y 0 & 1 & -x 0 \times \mathrm{v} 0 & -y 0 \times \mathrm{v} 0 \\
0 & 0 & 0 & x 1 & y 1 & 1 & -x 1 \times \mathrm{v} 1 & -y 1 \times \mathrm{v} 1 \\
0 & 0 & 0 & x 2 & y 2 & 1 & -x 2 \times \mathrm{v} 2 & -y 2 \times \mathrm{v} 2 \\
0 & 0 & 0 & x 3 & y 3 & 1 & -x 3 \times \mathrm{v} 3 & -y 3 \times \mathrm{v} 3
\end{array}\right) \cdot 
\left(
\begin{array}{l}
\mathrm{c} 00 \\
\mathrm{c} 01 \\
\mathrm{c} 02 \\
\mathrm{c} 10 \\
\mathrm{c} 11 \\
\mathrm{c} 12 \\
\mathrm{c} 20 \\
\mathrm{c} 21
\end{array}
 \right) =
 \left(
 \begin{array}{l}
\mathrm{u} 0 \\
\mathrm{u} 1 \\
\mathrm{u} 2 \\
\mathrm{u} 3 \\
\mathrm{v} 0 \\
\mathrm{v} 1 \\
\mathrm{v} 2 \\
\mathrm{v} 3
\end{array}
\right)
$$


where:
 $c_{ij}$ - matrix coefficients, $c_{22} = 1$

In [23]:
def ordered_points(points):
    ordered = np.zeros_like(points)
    x_plus_y = np.sum(points, axis=1)
    ordered[0] = points[np.argmin(x_plus_y)]
    ordered[2] = points[np.argmax(x_plus_y)]

    x_minus_y = np.diff(points, axis=1)
    ordered[1] = points[np.argmin(x_minus_y)]
    ordered[3] = points[np.argmax(x_minus_y)]

    return ordered



In [24]:
board_area = np.int32(ordered_points(corner_points))
cv2.fillConvexPoly(billboard,board_area,(0,0,0))
cv2.imshow('mask',billboard)
cv2.waitKey(0)
cv2.destroyAllWindows()

billboard_final = cv2.bitwise_or(billboard,warped_advert)
cv2.imshow('bill_with_advt', billboard_final)
cv2.waitKey(0)
cv2.destroyAllWindows()
